#  **AIFrenz GPU bootcamp**
##   End-to-End LLM finetune

by Hyungon Ryu | NVIDIA Sr. Solution Architect

본 컨텐츠는 NVIDIA OpenHackathon 팀이 운영하는 GPU Bootcamp contents를 Google Colab에 맞게 변경하였기 때문에, 몇몇 수동 설정 및 코드 변경하는 스크립트들이 존재합니다. 오리지널 컨텐츠는 다음의 링크를 통해 확인하실 수 있습니다. [End-to-End-LLM](https://github.com/openhackathons-org/End-to-End-LLM)

### GPU 환경 확인
`$ !nvidia-smi` 명령을 통해 현재 설치되어 있는 GPU를 확인할 수 있습니다.
아래와 같이 GPU 종류, 메모리 크기, 설치된 드라이버 정보 등

```
-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+------------------------+----------------------+
                                                                                         
+-----------------------------------------------------------------------------------------+
| Processes:                                                                              |
|  GPU   GI   CI        PID   Type   Process name                              GPU Memory |
|        ID   ID                                                               Usage      |
|=========================================================================================|
|  No running processes found                                                             |
+-----------------------------------------------------------------------------------------+
```


In [1]:
!nvidia-smi

Wed Jul  2 06:36:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 실습에 필요한 라이브러리 설치
LLM finetune을 위해서 hugginface transformer, datasets, peft, trl을 설치합니다.
quantization을 위해서 bitsandbytes를 설치합니다.
모델 파라미터, 데이터셋을 다운로드하기 위해서 google gdown을 설치합니다.
설치는 네트웍 환경에 따라 약 5분 정도 시간이 소요됩니다.

colab환경에서 NEMO toolkit 설치는 시간이 오래 걸리므로 생략하였습니다.

In [2]:
 %%time
!pip install --upgrade pip
!pip install transformers datasets langdetect peft trl bitsandbytes gdown


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 143.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 155.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 149.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 149.3 MB/s eta 0:00:00
   ━━━━━━━━━━━

## 실습에 필요한 utility 파일들을 라운로드합니다.
모두 OpenHackathon팀이 제작한 GPU Bootcamp 컨텐츠를 사용합니다.  

In [9]:

import gdown

os.makedirs("source", exist_ok=True)
scripts = {
    "source/download-guanco-ds.py": "https://github.com/openhackathons-org/End-to-End-LLM/raw/refs/heads/main/workspace-llm-use-case/source_code/Llama2/download-guanco-ds.py",
    "source/download-llama2-chat.py": "https://github.com/openhackathons-org/End-to-End-LLM/raw/refs/heads/main/workspace-llm-use-case/source_code/Llama2/download-llama2-chat.py",
    "source/download-llama2.py":"https://github.com/openhackathons-org/End-to-End-LLM/raw/refs/heads/main/workspace-llm-use-case/source_code/Llama2/download-llama2.py",
    "source/convert_llama_weights_to_hf.py":"https://github.com/openhackathons-org/End-to-End-LLM/raw/refs/heads/main/workspace-llm-use-case/source_code/Llama2/llama/convert_llama_weights_to_hf.py",
    "source/configuration_llama.py":"https://github.com/openhackathons-org/End-to-End-LLM/raw/refs/heads/main/workspace-llm-use-case/source_code/Llama2/llama/configuration_llama.py",
    "source/modeling_flax_llama.py":"https://github.com/openhackathons-org/End-to-End-LLM/raw/refs/heads/main/workspace-llm-use-case/source_code/Llama2/llama/modeling_flax_llama.py",
    "source/modeling_llama.py":"https://github.com/openhackathons-org/End-to-End-LLM/raw/refs/heads/main/workspace-llm-use-case/source_code/Llama2/llama/modeling_llama.py",
    "source/tokenization_llama.py":"https://github.com/openhackathons-org/End-to-End-LLM/raw/refs/heads/main/workspace-llm-use-case/source_code/Llama2/llama/tokenization_llama.py",
    "source/tokenization_llama_fast.py":"https://github.com/openhackathons-org/End-to-End-LLM/raw/refs/heads/main/workspace-llm-use-case/source_code/Llama2/llama/tokenization_llama_fast.py",


}

for fname, url in scripts.items():
    print(f"Downloading {fname} …")
    gdown.download(url, fname, quiet=False)


Downloading...
From: https://github.com/openhackathons-org/End-to-End-LLM/raw/refs/heads/main/workspace-llm-use-case/source_code/Llama2/download-guanco-ds.py
To: /content/source/download-guanco-ds.py


518B [00:00, 1.57MB/s]                   


Downloading...
From: https://github.com/openhackathons-org/End-to-End-LLM/raw/refs/heads/main/workspace-llm-use-case/source_code/Llama2/download-llama2-chat.py
To: /content/source/download-llama2-chat.py
267B [00:00, 713kB/s]                    


Downloading...
From: https://github.com/openhackathons-org/End-to-End-LLM/raw/refs/heads/main/workspace-llm-use-case/source_code/Llama2/download-llama2.py
To: /content/source/download-llama2.py
262B [00:00, 474kB/s]                    


Downloading...
From: https://github.com/openhackathons-org/End-to-End-LLM/raw/refs/heads/main/workspace-llm-use-case/source_code/Llama2/llama/convert_llama_weights_to_hf.py
To: /content/source/convert_llama_weights_to_hf.py
13.3kB [00:00, 21.1MB/s]                   


Downloading...
From: https://github.com/openhackathons-org/End-to-End-LLM/raw/refs/heads/main/workspace-llm-use-case/source_code/Llama2/llama/configuration_llama.py
To: /content/source/configuration_llama.py
9.39kB [00:00, 11.5MB/s]                   


Downloading...
From: https://github.com/openhackathons-org/End-to-End-LLM/raw/refs/heads/main/workspace-llm-use-case/source_code/Llama2/llama/modeling_flax_llama.py
To: /content/source/modeling_flax_llama.py
30.1kB [00:00, 48.2MB/s]                   


Downloading...
From: https://github.com/openhackathons-org/End-to-End-LLM/raw/refs/heads/main/workspace-llm-use-case/source_code/Llama2/llama/modeling_llama.py
To: /content/source/modeling_llama.py
65.4kB [00:00, 60.7MB/s]                   


Downloading...
From: https://github.com/openhackathons-org/End-to-End-LLM/raw/refs/heads/main/workspace-llm-use-case/source_code/Llama2/llama/tokenization_llama.py
To: /content/source/tokenization_llama.py
22.0kB [00:00, 24.8MB/s]                   


Downloading...
From: https://github.com/openhackathons-org/End-to-End-LLM/raw/refs/heads/main/workspace-llm-use-case/source_code/Llama2/llama/tokenization_llama_fast.py
To: /content/source/tokenization_llama_fast.py
13.1kB [00:00, 37.1MB/s]                   


## 실습 모델 다운로드
실습에 필요한 Llama-2 checkpoint를 다운로드 받습니다.
Huggingface 사이트에서 직접 받는 것도 가능합니다.
*주의*
용량이 큰 2개의 파일을 받기 때문에 약 10분 이상의 시간이 걸립니다.
다운로드 후 런타임을 다시 시작하면 다시 받으셔야 할 수 있습니다.  
Google Drive를 연동하는 경우 복사해 놓으시면 다운로드한 이미지를 계속 사용할 수 있습니다.

In [4]:
#   Download Llama-2 checkpoints via gdown
import gdown
import os

os.makedirs("model", exist_ok=True)
links = {
    "model/Llama-2-7b-chat.tar": "https://drive.google.com/uc?id=1rfMCxT8fUQwQ5Pe6z7kDgK3nAZ6FyEeT&confirm=t",
}

for out, url in links.items():
    print(f"Downloading {out} …")
    gdown.download(url, out, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1rfMCxT8fUQwQ5Pe6z7kDgK3nAZ6FyEeT&confirm=t
To: /content/model/Llama-2-7b-chat.tar
100%|██████████| 27.0G/27.0G [03:15<00:00, 138MB/s]


Downloading...
From: https://drive.google.com/uc?id=1kRyLpeg1ur8HC-wgsskR2dPmpmvL8qi6&confirm=t
To: /content/model/Llama-2-7b.tar
100%|██████████| 27.0G/27.0G [03:11<00:00, 141MB/s]


하드디스크를 절약하기 위해서 압축을 푼 후 압축 파일을 삭제합니다.

In [5]:
 %%time
!tar -xf model/Llama-2-7b-chat.tar -C model && rm model/Llama-2-7b-chat.tar


^C
CPU times: user 948 ms, sys: 150 ms, total: 1.1 s
Wall time: 3min 34s


## Inference 테스트


### *주의*
Hugginface transformer pipeline을 이용하기 위해서는 다운로드한 체크포인트를
Hugginface Transformer 라이브러리에 맞게 변환을 해줘야합니다.
변환 및 저장에 약 3분 정도의 시간이 소요됩니다.

In [10]:
%%time
!python source/convert_llama_weights_to_hf.py \
    --input_dir model/Llama-2-7b-chat \
    --model_size 7B \
    --output_dir model/Llama-2-7b-chat-hf

2025-07-02 07:07:32.998548: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751440053.018031    9341 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751440053.024183    9341 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://gith

In [13]:
!nvidia-smi

Wed Jul  2 07:11:26 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P0             27W /   70W |   14928MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## chat prompt 테스트를 진행합니다.

HF pipeline을 통해서 간단한 prompt를 실행해보는 예제입니다.

In [12]:

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

BASE = "model/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(BASE, trust_remote_code=True)
model     = AutoModelForCausalLM.from_pretrained(BASE)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=100)
out = pipe("<s>[INST] What is astrophotography? [/INST]")[0]["generated_text"]
print(out)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 166.12 MiB is free. Process 12237 has 14.58 GiB memory in use. Of the allocated memory 14.48 GiB is allocated by PyTorch, and 1.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## 4-bit 양자화


Several ways and algorithms to quantize a model including can be found [here](https://huggingface.co/docs/peft/main/en/developer_guides/quantization). A library to easily implement quantization and integrate with transformers is the `bitsandbytes` library. The library provides config parameters to quantize a model to 8 or 4 bits using the `BitsAndBytesConfig` class. The 4 bits parameters used in the cell below are described as follows:

- **load_in_4bit**: set `True` to quantize the model to 4-bits when you load it
- **bnb_4bit_quant_type**: set to `"nf4"` to use a special 4-bit data type for weights initialized from a normal distribution
- **bnb_4bit_use_double_quant**: set `True` to use a nested quantization scheme to quantize the already quantized weights
- **bnb_4bit_compute_dtype**: set to `torch.float16` or `torch.bfloat16` to use bfloat16 for faster computation

Run the cell below to set the 4-bit quantization for our model.

## **주의**
메모리 이슈로 인해 런타임을 재시작을 하는 경우  **`세션 다시시작`**을 누르시면 됩니다.
절대로 런타임 연결해제를 누르면 안됩니다. 설치된 라이브러리와 다운로드한 체크포인트, 데이터가 모두 사라집니다.


In [1]:
import os
import torch
import json
from datasets import load_dataset, load_from_disk
from langdetect import detect
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
import re

import warnings
warnings.filterwarnings("ignore")

15GB 메모리 내에서 inference /finetune을 진행하기 위해서
bitsandbytes를 활성화해야합니다.

In [2]:

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

In [4]:
BASE = "model/Llama-2-7b-chat-hf"

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    BASE,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
!nvidia-smi

Wed Jul  2 07:15:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P0             27W /   70W |    6676MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### tokenizer를 로드합니다.

In [7]:
tokenizer = AutoTokenizer.from_pretrained(BASE, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

### HF pipeline을 정의합니다.

In [10]:
def run_inference(prompt,max_length=200 ):
    inf_pipeline = pipeline(model=model, tokenizer=tokenizer, max_length=max_length, task="text-generation")
    prompt = f"<s>[INST] {prompt} [/INST]"
    result = inf_pipeline(prompt)
    print(result[0]['generated_text'])

In [11]:
prompt = "explain what is astrophotography?"
run_inference(prompt)

Device set to use cuda:0


<s>[INST] explain what is astrophotography? [/INST]  Astrophotography is a type of photography that involves capturing images of celestial objects such as stars, planets, galaxies, and other astronomical phenomena. Hinweis auf die Bildgröße der Sonne und des Mondes. Astrophotography can be done using a variety of techniques and equipment, including:

1. DSLR cameras: These are the most common type of camera used for astrophotography. They have large sensors that can capture a lot of light, making it possible to capture detailed images of celestial objects.
2. Mirrorless cameras: These cameras are similar to DSLRs but have a more compact design and are often more lightweight, making them easier to use for astrophotography.
3. Cameras with full-frame sensors: These cameras have larger sensors than


In [12]:
!nvidia-smi

Wed Jul  2 07:18:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P0             27W /   70W |    6760MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Finetune llama2 model with custom dataset

## 데이터 전처리 (Data Preprocessing)

We consider [openassistant-guanaco](https://huggingface.co/datasets/timdettmers/openassistant-guanaco) as our choice dataset for finetuning. The openassistant-guanaco dataset is a subset of the [Open Assistant Conversations](https://huggingface.co/datasets/OpenAssistant/oasst1/tree/main) dataset that contains only the highest-rated paths in the conversation tree, with a total of 9,85k training samples and 518 test samples. The OpenAssistant Conversations (OASST1) is a human-generated, human-annotated assistant-style conversation corpus consisting of 161,443 messages in 35 different languages, annotated with 461,292 quality ratings, resulting in over 10,000 fully annotated conversation trees. The corpus is a product of a worldwide crowd-sourcing effort involving over 13,500 volunteers ([source](https://huggingface.co/datasets/OpenAssistant/oasst1)).

### Dataset Structure

Each row in the `openassistant-guanaco` jsonl dataset is a text dictionary that consists of `human` instructions and `Assistant` that provide context to draw the response to the instruction. Within the `text`, the `human` and `Assistant` fields are separated with three `###` delimiters that denote the start and end of positions.  

```bash

{ "text": "### Human: Can you write a short introduction about the relevance of the term \"monopsony\" in economics? Please use examples related to...
           ### Assistant: \"Monopsony\" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term  is...   
           ### Human: Now explain it to a dog"
}
{ "text": "### Human: \u00bfCUales son las etapas del desarrollo y en qu\u00e9 consisten seg\u00fan Piaget?
           ### Assistant: Jean Piaget fue un psic\u00f3log suizo que propuso una teor\u00eda sobre el desarrollo cognitivo...
           ### Human: \u00bfHay otras teor\u00edas sobre las etapas del desarrollo que reafirmen o contradigan a la teor\u00eda de Piaget?"
}
{"text": "### Human: Can you give me an example of a python script that opens an api point and serves a string?
          ### Assistant: Sure! Here's an example Python script that uses the Flask web framework to create a simple API endpoint that serves a string:\n\n`         \nfrom flask import Flask\n\napp = Flask(__name__)\n\n@app.route('/')\ndef hello_world():\n    return 'Hello, world!'\n\nif __name__ ==  \n ...   
          ### Human: What changes would you need to make to the code above to serve a JSON object instead of a string?
          ### Assistant: To serve a JSON object instead of a string, you can modify the \"hello_world()\" function to return a JSON response using the  Flask \"jsonify\" function. Here's an example of how to modify the previous code to serve a JSON object:\n\n... "
}
...

```


Please run the cell below to check if the dataset exists in the data directory; otherwise, uncomment the nested cell below to download it.

## *주의*
소스코드를 확인하여 데이터 저장위치를 Colab환경에 맞게 다음과 같이 조정해주세요
`/content/source/download-guanco-ds.py`

변경전
```
output_eva = '../data/openassistant_best_replies_eval.jsonl'
output_train = '../data/openassistant_best_replies_train.jsonl'

```

변경후
```
output_eva = '/content/data/openassistant_best_replies_eval.jsonl'
output_train = '/content/data/openassistant_best_replies_train.jsonl'

```

In [16]:
###### download dataset. Remove comment to run the cell ###########
import os

os.makedirs("data", exist_ok=True)
%time !python3 source/download-guanco-ds.py

Downloading...
From: https://drive.google.com/uc?id=1PFLTYBJi0rLQlFjWrr8ioaIgoNbiffXx&confirm=t
To: /content/data/openassistant_best_replies_eval.jsonl
100% 1.11M/1.11M [00:00<00:00, 183MB/s]
Downloading...
From: https://drive.google.com/uc?id=1tAJI0z_dyZAX9MW6tYmv3znH_561BDVY&confirm=t
To: /content/data/openassistant_best_replies_train.jsonl
100% 20.9M/20.9M [00:00<00:00, 77.0MB/s]
CPU times: user 110 ms, sys: 21.5 ms, total: 132 ms
Wall time: 25.8 s


In [18]:
!ls -alh data

total 21M
drwxr-xr-x 2 root root 4.0K Jul  2 07:28 .
drwxr-xr-x 1 root root 4.0K Jul  2 07:26 ..
-rw-r--r-- 1 root root 1.1M Feb 12  2024 openassistant_best_replies_eval.jsonl
-rw-r--r-- 1 root root  20M Feb 12  2024 openassistant_best_replies_train.jsonl


**Expected Output:**
```python
data:
README.md  openassistant_best_replies_eval.jsonl   simple_data.json
filtered   openassistant_best_replies_train.jsonl
...

```


Import all required libraries

In [2]:
# In some cases where you have access to limited computing resources, you might have to uncomment os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64" if you run into not enough memory issue


import os
import torch
import json
from datasets import load_dataset, load_from_disk
from langdetect import detect
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
import re
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

import warnings
warnings.filterwarnings("ignore")

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64"

### Data Extraction

Let's execute the `read_jsonl` function below to read the dataset

In [20]:

def read_jsonl(file_path):
    with open(file_path) as f:
        data = [json.loads(line) for line in f]
        return data


Let's perform the following steps in the cell below:
- Set the path to the train and test jsonl files
- Read both files using the `read_jsonl` function
- Extract 5k samples from the training set
- Display the samples to see the content and format

In [21]:
#set path to jsonl files
train_path = '/content/data/openassistant_best_replies_train.jsonl'
test_path = '/content/data/openassistant_best_replies_eval.jsonl'

# read the files
raw_train_data = read_jsonl(train_path)
raw_test_data = read_jsonl(test_path)

# extract 5000 samples
train_samples = raw_train_data[:5000]

print("length of traning samples: ", len(train_samples))
train_samples[:10]

length of traning samples:  5000


[{'text': '### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining p

In [22]:
print( len(raw_train_data), len(train_samples) )

9846 5000


**Expected output:**
```python

length of traning samples:  5000
[{'text': '### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as ..'},
 ...
 {'text': '### Human: ¿CUales son las etapas del desarrollo y en qué consisten según Piaget?### Assistant: Jean Piaget fue un psicólogo suizo que propuso una teoría sobre el desarrollo cognitivo humano que consta de cuatro etapas:\n\nEtapa sensoriomotora (0-2 años): Durante esta etapa, el niño aprende a través de sus sentidos y movimientos. Descubre que sus acciones pueden tener un impacto en el entorno y comienza a formarse una idea básica de objetividad y continuidad.\n\nEtapa preoperatoria (2-7 años): En esta etapa, el niño comienza a desarrollar un pensamiento simbólico y a comprender que las cosas pueden representar a otras cosas. También comienzan a desarrollar un pensamiento lógico y a comprender conceptos como la causa y el efecto.\n\nEtapa de ...?'},
 ...
 {'text': '### Human: Schreibe einen kurze und präzise Konstruktionsbeschreibung zu einem Dreieck ABC mit c=6\xa0cm, h_c=5\xa0cm und γ=40°. γ ist hierbei der von Seite c gegenüberliegende Winkel.### Assistant: Dreieck ABC ist ein rechtwinkliges Dreieck mit der Seitenlänge c=6 cm als Hypotenuse. Die Höhe h_c von c beträgt 5 cm und der Winkel γ von c gegenüberliegend beträgt 40°.### Human: Vielen Dank, das hat mir sehr weitergeholfen.'},
 {'text': '### Human: Напиши информацию о игре Hytale### Assistant: Hytale - это предстоящая игра-песочница, придуманная Hypixel Studios. Объединяя в себе диапазон игры-песочницы с глубиной ролевой игры, Hytale погружает игроков в живой космос, полный эскапад и творчества.\n\nВ Hytale геймеры могут путешествовать по просторному, процедурно созданному миру и принимать участие в разнообразных действиях. От возведения баз и создания собственных персонажей до создания карт приключений и сражений с монстрами или другими игроками - возможности Hytale безграничны. Игроки также могут создавать свои ..'},
 {'text': '### Human: 私は猫ちゃんが好きなんですけど\n貴方も猫ちゃんが好きですか?### Assistant: 猫ちゃんが好きなんですね。\n私も猫ちゃんが好きですよ！\n猫ちゃんのどんなところが好きですか？### Human: 猫ちゃんの全てが好きです！猫ちゃんは天使です！！### Assistant: わかりますよ、その気持ち！\n猫ちゃんの愛くるしい姿は、天使みたいに可愛いですよね！\nあなたのお家に、猫ちゃんはいるんですか？### Human: 勿論です！とっても可愛くて、毎日が幸せです！'},

 {'text': "### Human: Quins sinònims amb altres dialectes d'aquesta llengua té nen o nena?### Assistant: Al·lot o al·lota, vailet o vaileta, manyac o manyaga, nin o nina, xiquet o xiqueta, xic o xica, marrec, minyó o minyona."},
...
```

We can see from the content displayed that the file contains text written in different languages. For demonstration purposes, we want to consider only English text for training and prompting. Therefore, the train and test sets will be processed to filter out non-English text samples using a `detect` feature from the `spacy-langdetect` library. The English-only texts (`1778 training samples`) are saved as the new training set. The same process is applied to the test samples. Run the three two cells below to execute the filter process.     

In [27]:
# filter non-Emglish word
def remove_nonEnglish_rows(ds):
    new_ds = []
    for row in (ds):
        if detect(str(row)) == 'en':
            new_ds.append(row)
    return new_ds

# save English text samples
os.makedirs("/content/data/filtered", exist_ok=True)

def save_jsonl(ds,filename):
    with open(f"/content/data/filtered/{filename}.jsonl", "w") as write_file:
            json.dump(ds, write_file, indent=4)
            print("dataset saved in jsonl format ....")

In [24]:
#calling filter function
filter_train_samples = remove_nonEnglish_rows(train_samples)
filter_test_samples = remove_nonEnglish_rows(raw_test_data )

print("len of training samples: ",len(filter_train_samples))
filter_train_samples[:10]

len of training samples:  1778


[{'text': '### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining p

In [25]:
print( len(raw_train_data),  len(train_samples), len(filter_train_samples), )
print( len(raw_test_data),   len(filter_test_samples) )

9846 5000 1778
518 189


**Expected output:**
```python
len of training samples:  1775
[{'text': '### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as ...'},
 {'text': '### Human: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?### Assistant: Sure! Let\'s say you want to build a model which can distinguish between images of cats and dogs. You gather your dataset, consisting of many cat and dog pictures. Then you put them through a neural net of your choice, which produces some representation for each image, a sequence of numbers like [0.123, 0.045, 0.334, ...]. The problem is, if your model is unfamiliar with cat and dog images, these representations will be quite random. At one time a cat and ...'},
 {'text': "### Human: I want to start doing astrophotography as a hobby, any suggestions what could i do?### Assistant: Getting started in astrophotography can seem daunting, but with some patience and practice, you can become a master of the craft. To begin, you'll need a good camera and lens, a tripod, and a dark sky location free of light pollution. You will also need to learn about the basics of astrophotography, such as what camera settings to use, how to capture star trails, and the best techniques for tracking celestial objects. You can also purchase or rent different types of telescopes, depending on what types of objects you want to capture. Additionally, it's important to keep up with the latest astrophotography news and trends. Once you have the necessary ..."},
...
```

In [28]:
# set file names
save_train_filename = 'train'
save_test_filename = 'test'

# save file
save_jsonl(filter_train_samples, save_train_filename)
save_jsonl(filter_test_samples, save_test_filename)

dataset saved in jsonl format ....
dataset saved in jsonl format ....


### Data Transformation

In this section, we want to format our dataset to the Llama2 acceptable template for finetuning:

```python

<s>[INST] {human text} [/INST] {assistant/context} </s>
<s>[INST]{human text} [/INST] </s>

```
- **Human text**: It denotes human instructions to the model. The human text is enclosed within an instruction tag `[INST] [/INST]`
- **Assistant**: represents the context that will assist the model in drawing out a response to the instruction issued by a human. The assistant text is nested to the human text within a segment tag `<s>  </s>`.

One or more segments can exist within a training sample text. A segment can consist of both human instruction text and assistant context or human instruction text only.


<img src="images/template.png"/>


Next, we load the filtered dataset by running the cell below.

In [30]:
dataset = load_dataset('/content/data/filtered')

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Execute the function to transform the filtered dataset to the format explained above.

In [31]:
### credit: https://huggingface.co/datasets/mlabonne/guanaco-llama2-1k ###

# Define a function to transform the data
def transform_to_template(example):
    conversation_text = example['text']
    segments = conversation_text.split('###')

    reformatted_segments = []

    # Iterate over pairs of segments
    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace('Human:', '').strip()

        # Check if there is a corresponding assistant segment before processing
        if i + 1 < len(segments):
            assistant_text = segments[i+1].strip().replace('Assistant:', '').strip()

            # Apply the new template
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
        else:
            # Handle the case where there is no corresponding assistant segment
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

    return {'text': ''.join(reformatted_segments)}


# Apply the transformation
template_dataset = dataset.map(transform_to_template)

Map:   0%|          | 0/1778 [00:00<?, ? examples/s]

Map:   0%|          | 0/189 [00:00<?, ? examples/s]

In [32]:
print( len(template_dataset['train']))
print( len(template_dataset['test']))

1778
189


Let's display a sample to inspect if our training set is in the right format as shown in the screenshot above.

In [33]:
template_dataset['train'][2:3]

{'text': ["<s>[INST] I want to start doing astrophotography as a hobby, any suggestions what could i do? [/INST] Getting started in astrophotography can seem daunting, but with some patience and practice, you can become a master of the craft. To begin, you'll need a good camera and lens, a tripod, and a dark sky location free of light pollution. You will also need to learn about the basics of astrophotography, such as what camera settings to use, how to capture star trails, and the best techniques for tracking celestial objects. You can also purchase or rent different types of telescopes, depending on what types of objects you want to capture. Additionally, it's important to keep up with the latest astrophotography news and trends. Once you have the necessary equipment and knowledge, you can start shooting and experimenting with different techniques to get the best results. </s><s>[INST] Can you tell me more?  What would you recommend as a basic set of equipment to get started with?  H

Save the preprocessed dataset to the directory `../data/ds_preprocess`.

In [34]:
os.makedirs("/content/data/ds_preprocess", exist_ok=True)
template_dataset.save_to_disk('/content/data/ds_preprocess')

Saving the dataset (0/1 shards):   0%|          | 0/1778 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/189 [00:00<?, ? examples/s]

## Fine-tuning LLAMA 2

As mentioned earlier in the notebook, our choice model for finetuning is the `Llama-2-7b-chat`. Below is the list of walkthrough steps to complete the task.

- Convert the `Llama-2-7b-chat` to Hugging Face transformer format
- Set the paths to the model and load the dataset
- Load the model tokenizer
- Set the training parameter
- Configure the Parameter Efficient Fine Tuning with LoRA
- Apply 4-bits quantization
- Setup the trainer to start the finetuning process


### Convert model to Hugging Face Transformers Format
We will convert our model checkpoint to the Hugging Face transformer format. The benefits are as follows:  

- Delivers a convenient way to finetune the Llama2 model with limited GPU computing resources (like laptops, workstations, or Google Colab) through the use of some technique that a transformer-compatible
- Ability to immediately use a model on a given input text using transformer Pipelines.
- The use of transformers `Pipelines` group together the pre-trained model with the preprocessing that was used during that training to enable quick inferencing.


To convert our model checkpoint, we use the `convert_llama_weights_to_hf.py` script located on [GitHub](https://github.com/cedrickchee/transformers-llama/tree/llama_push/src/transformers/models/llama).

- `--input_dir`: denotes the directory to the llama model to be converted
- `--model_size`: represents the llama model parameter size
- `--out_dir`: the directory to save the converted model

<img src="images/llama-hf.png" height="550px" width="900px" />

If you already have the Hugging Face format of the model `Llama-2-7b-chat-hf`, you can skip running the cell below.



In [ ]:
%time !python3 ../../source_code/Llama2/llama/convert_llama_weights_to_hf.py \
--input_dir ../../model/Llama-2-7b-chat --model_size 7B --output_dir ../../model/Llama-2-7b-chat-hf

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Fetching all parameters from the checkpoint at ../../model/Llama-2-7b-chat.
Loading the checkpoint in a Llama model.
Loading checkpoint shards: 100%|████████████████| 33/33 [00:06<00:00,  4.85it/s]
Saving in the Transformers format.


**Exepected Output:**
```python
...
https://github.com/huggingface/transformers/pull/24565
Fetching all parameters from the checkpoint at ../model/Llama-2-7b-chat.
Loading the checkpoint in a Llama model.
Loading checkpoint shards: 100%|████████████████| 33/33 [00:06<00:00,  4.95it/s]
Saving in the Transformers format.

```

## **주의**
메모리 이슈로 인해 런타임을 재시작을 하는 경우  **`세션 다시시작`**을 누르시면 됩니다.
절대로 런타임 연결해제를 누르면 안됩니다. 설치된 라이브러리와 다운로드한 체크포인트, 데이터가 모두 사라집니다.


In [3]:
# In some cases where you have access to limited computing resources, you might have to uncomment os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64" if you run into not enough memory issue


import os
import torch
import json
from datasets import load_dataset, load_from_disk
from langdetect import detect
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
import re
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

import warnings
warnings.filterwarnings("ignore")

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64"

Now, we can initialize the path to our transformer format model and load the transformed/preprocessed training dataset from the directory where it was saved.

In [4]:

# initailize path to the base model
base_model = "/content/model/Llama-2-7b-chat-hf"

# set the path to the dataset template
data_path = "/content/data/ds_preprocess/train"

# set the path to the dataset template
eval_path = "/content/data/ds_preprocess/test"

# load the transformed dataset
train_ds = load_from_disk(data_path)
eval_ds = load_from_disk(eval_path)

### Loading tokenizer

Tokenization is breaking a text into sentences, words, or sub-words. Each word or sentence in a text is considered as a token. Tokenization allows a detailed text data analysis when broken into smaller units. The [LLaMA tokenizer](https://huggingface.co/docs/transformers/en/model_doc/llama2) is a byte-pair-encoding (BPE) model based on [sentencepiece](https://aclanthology.org/P16-1162/), an unsupervised text tokenizer and detokenizer for Neural Network-based text generation systems that predetermined the vocabulary size prior to the neural model training.

In the cell below, we load the tokenizer from our base model directory and set the parameters:

- **pad_token**: a special token used to make arrays of tokens the same size for batching purposes.
- **padding_side**: side to pad

A comprehensive list of Llama tokenizer parameters can be found [here](https://huggingface.co/docs/transformers/en/model_doc/llama2)

In [5]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

### Set Training Hyperparameters

Training hyperparameters are customized using the `TrainingArguments` class. The class provides an API that offers a wide range of options to customize and optimize the training process. Please find a comprehensive description of the hyperparameters [here](https://huggingface.co/transformers/v3.0.2/main_classes/trainer.html#transformers.TrainingArguments). You can further modify the values of the hyperparameters in the next cell after the complete finetune process and rerun the cells to see how it impact on the training outcome.

### Configure PEFT With LoRA

Finetuning large language pre-trained models is computationally costly. Our main goal is to accelerate our finetuning process with minimal memory consumption. A method to achieve that is to use a state-of-the-art [Parameter-Efficient Finetuning (PEFT)](https://github.com/huggingface/peft/tree/main) approach. [PEFT](https://arxiv.org/abs/2305.16742) allows finetuning a small number of (extra) model parameters instead of all the model's parameters, and this significantly decreases the computational and storage costs. One of the ways to implement PEFT is to adopt the Low-Rank Adaptation (LoRA) technique. Lora makes finetuning more efficient by greatly reducing the number of trainable parameters for downstream tasks. It does this by freezing the pre-trained model weights and injecting trainable rank decomposition matrices into each layer of the Transformer architecture. According to the [authors of LoRA](https://arxiv.org/abs/2106.09685), Aside from reducing the number of trainable parameters by 10k times, it also reduces the GPU consumption by 3x, thus delivering high throughput with no inference latency. For quick background on LoRA, please follow this [link](https://huggingface.co/docs/peft/main/en/conceptual_guides/lora).

<center><img src="images/lora-arch.png" height="500px" width="900px"  /></center>
<center>  LoRA reparametrization and Weight merging. <a href="https://huggingface.co/docs/peft/main/en/conceptual_guides/lora"> View source</a> </center>

LoRA techniques are applied through `LoraConfig`, which provides PEFT parameters that control how the method is applied to the base model. A description of the parameter used in the cell below is given as follows:

- **lora_alpha**: LoRA scaling factor
- **lora_dropout**: The dropout probability for LoRA layers.
- **r**: the rank of the update matrices, expressed in int. Lower rank results in smaller update matrices with fewer trainable parameters.
- **bias**: Specifies if the bias parameters should be trained. It can be 'none', 'all', or 'lora_only'.
- **task_type**: Possible task types which include `CAUSAL_LM`, `FEATURE_EXTRACTION`, `QUESTION_ANS`, `SEQ_2_SEQ_LM`, and `SEQ_CLS and TOKEN_CLS`.   

Because the task we want to perform is text generation, we have set the task_type to Causal language model `(CAUSAL_LM)`, which is frequently used for text generation tasks. Please run the cell below to set up the LoRA configuration.

In [6]:
lora_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

### 4-bit quantization configuration

Model quantization is a popular deep-learning optimization method in which model data—network parameters and activations—are converted from floating-point to lower-precision representation, typically using 8-bit integers. Quantization represents data with fewer bits, making it a useful technique for reducing memory usage and accelerating inference, especially in large language models (LLMs). It can be combined with PEFT methods to make it easier to train and load LLMs for inference.

<center><img src="images/quantization.png" height="400px" width="700px" /></center>
<center> <a href="https://developer.nvidia.com/blog/achieving-fp32-accuracy-for-int8-inference-using-quantization-aware-training-with-tensorrt/" > source: Using Quantization Aware Training with NVIDIA TensorRT</a></center>

Several ways and algorithms to quantize a model including can be found [here](https://huggingface.co/docs/peft/main/en/developer_guides/quantization). A library to easily implement quantization and integrate with transformers is the `bitsandbytes` library. The library provides config parameters to quantize a model to 8 or 4 bits using the `BitsAndBytesConfig` class. The 4 bits parameters used in the cell below are described as follows:

- **load_in_4bit**: set `True` to quantize the model to 4-bits when you load it
- **bnb_4bit_quant_type**: set to `"nf4"` to use a special 4-bit data type for weights initialized from a normal distribution
- **bnb_4bit_use_double_quant**: set `True` to use a nested quantization scheme to quantize the already quantized weights
- **bnb_4bit_compute_dtype**: set to `torch.float16` or `torch.bfloat16` to use bfloat16 for faster computation

Run the cell below to set the 4-bit quantization for our model.

In [7]:

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

### Loading Base Model

The next step is to load our base model `(Llama-2-7b-chat-hf)` with the causal language model class used for the text generation task. We do this by passing the base model, quantization config, and GPU device ID to the `AutoModelForCausalLM` object, as shown in the cell below

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
!nvidia-smi

Wed Jul  2 08:08:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P0             27W /   70W |    6676MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Set the Trainer Hyperparameters

To initiate our model trainer, we create a trainer object from [Supervised fine-tuning (SFT)](https://huggingface.co/docs/trl/en/sft_trainer). SFT is part of the integrated transformer [Reinforcement Learning (TRL)](https://huggingface.co/docs/trl/en/index) tools used to train transformer language models using Reinforcement Learning. Others include [Reward Modeling step (RM)](https://huggingface.co/docs/trl/en/reward_trainer) and  Proximal [Policy Optimization (PPO)](https://arxiv.org/abs/1707.06347). In our SFT trainer object, we set our model, training dataset, PEFT config  object, model tokenizer, and training argument parameter. We also specify the field (`text`) to use within our dataset.

**Note:** *If running on a single DGX A100 GPU, modify the value of `max_seq_length` to 1024 or set it to none (as default).*

In [12]:
from trl import SFTConfig, SFTTrainer


sft_cfg = SFTConfig(
    # — data & formatting
    dataset_text_field="text",   # which column to read
    max_seq_length=128,          # truncate / pad length

    # — training schedule
    num_train_epochs=1,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,

    # — optimization
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    weight_decay=0.001,
    lr_scheduler_type="constant",
    warmup_ratio=0.03,
    max_grad_norm=0.3,

    # — precision & logging
    bf16=True,
    fp16=False,
    save_steps=25,
    logging_steps=1,

    # — output & reporting
    output_dir="/content/model/results",
    save_total_limit=3,
    run_name=None,               # no default W&B run name
    report_to=["tensorboard"],   # ONLY TensorBoard, no "wandb"

    # — packing
    packing=False,               # example‐packing off
    eval_packing=False,
)


average_tokens_across_devices is set to True but it is invalid when world size is1. Turn it to False automatically.


In [ ]:

trainer = SFTTrainer(
    model,                   #  4-bit + LoRA model
    train_dataset=train_ds,  #  HF train dataset
    args=sft_cfg,            # 3 SFTConfig object
    eval_dataset=eval_ds,    #   eval dataset
    peft_config=lora_config,   #   LoRA settings
    processing_class=tokenizer,  # pass tokenizer
)

trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
1,2.693700
2,2.601900
3,2.609800
4,2.589400
5,2.441700
6,2.327700
7,2.022700
8,2.330100
9,2.024500
10,1.888800


Run the cell below to train the SFT trainer object

Save the finetuning model and tokenizer in the directory  `/content/model/Llama-2-7b-chat-hf-finetune`

In [ ]:
# save model
new_model = "/content/model/Llama-2-7b-chat-hf-finetune"
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

## Inferencing

Now that we completed our finetuning process, we can run a quick inference to test how our new model is performing. To do that, we will create the following:

- a transformer pipeline with four parameter inputs: `model`, `tokenizer`, `max_length`, and `task`.  
- format prompt as: `f"<s>[INST] {prompt} [/INST]"`
- pass the prompt into the pipeline object and get result.

```python

inf_pipeline = pipeline(model=model, tokenizer=tokenizer, max_length=200, task="text-generation")
prompt = inf_pipeline(f"<s>[INST] {prompt} [/INST]")
result = inf_pipeline(prompt)
print(result[0]['generated_text'])
```
You can modify the `max_length` to decide the length of text generated by the model.


In [ ]:

def run_inference(prompt):
    inf_pipeline = pipeline(model=model, tokenizer=tokenizer, max_length=128, task="text-generation")
    prompt = f"<s>[INST] {prompt} [/INST]"
    result = inf_pipeline(prompt)
    print(result[0]['generated_text'])

In [ ]:
prompt = "explain what is astrophotography?"
run_inference(prompt)

**Likely output:**

```bash
<s>[INST] explain what is astrophotography? [/INST] Astrophotography is the branch of photography that deals with the photographing of celestial objects such as stars, planets, galaxies, and other astronomical phenomena. Astrophotographers use specialized cameras and telescopes to capture images of these objects, often in low light conditions. Astrophotography requires a great deal of patience, skill, and knowledge, as photographers must be able to accurately track the movement of celestial objects and compensate for the effects of light pollution and atmospheric distortion. Astrophotography has become increasingly popular in recent years, with many amateur astronomers and professional photographers engaging in this hobby and art form.

Astrophotography can be done using a variety of techniques, including long exposure times, tracking mounts, and using narrow-band filters to
```

In [ ]:
prompt = "can you explain further?"
run_inference(prompt)

**Likely output:**

```bash
<s>[INST] can you explain further? [/INST] Sure, I'd be happy to explain further! Could you please provide more context or clarify what you're asking about? 😃 👍 💡 🖊 📝 📅 🕰 🕰️ 🕳️ 🕷️ 🕸️ 🕷️ 🕸️ 🕳️ 🕷️ 🕸️ 🕳️ 🕷️ 🕸️ 🕰️ 🕰️ 🕳️ 🕷️ 🕸️ 🕳️ 🕷️ 🕸️ 🕰️

```

In [ ]:
prompt = "I want you to explain further on astrophotography"
run_inference(prompt)

**Likely output:**

```bash

<s>[INST] I want you to explain futher on astrophotography [/INST] Sure, I d be happy to explain further on astrophotography. Astrophotography is the process of capturing images of celestial objects such as stars, planets, galaxies, and other astronomical phenomena. It involves using specialized cameras and techniques to capture high-quality images of these objects in the night sky.

There are several key components to astrophotography:

    Camera: Astrophotography cameras are designed specifically for capturing images of celestial objects. They typically have large sensors, fast lenses, and specialized features such as built-in equatorial mounts, motorized tracking systems, and cooling systems to reduce noise and improve image quality.

    Telescope: A telescope is used to gather light from the celestial object being photographed. There are several types of

```

### Reload model in FP16 and merge with LoRA weights

To have our model as a single entity for ease of use and widened task coverage, we reload it in fp16 mode and merge it with the LoRA weights using `model.merge_and_unload()`. The tokenizer is reloaded, pad, and saved along with the merged model in the same directory, `/content/model/Llama-2-7b-chat-hf-merged`.

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
load_base_model = AutoModelForCausalLM.from_pretrained( base_model, torch_dtype=torch.float16, low_cpu_mem_usage=True, return_dict=True, device_map={"": 0})

model = PeftModel.from_pretrained(load_base_model, new_model)
model = model.merge_and_unload()


# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


In [ ]:
model.save_pretrained("/content/model/Llama-2-7b-chat-hf-merged", safe_serialization=True)
tokenizer.save_pretrained("/content/model/Llama-2-7b-chat-hf-merged")

**expected output**:
```python
('/content/model/Llama-2-7b-chat-hf-merged/tokenizer_config.json',
 '/content/model/Llama-2-7b-chat-hf-merged/special_tokens_map.json',
 '/content/model/Llama-2-7b-chat-hf-merged/tokenizer.model',
 '/content/model/Llama-2-7b-chat-hf-merged/added_tokens.json',
 '/content/model/Llama-2-7b-chat-hf-merged/tokenizer.json')
```


# additional work for meerkat model

meerkat is medical domain sllm model from mistal-8b from Korea University.

[Meerkat-7B-v1.0](https://huggingface.co/dmis-lab/meerkat-7b-v1.0)is an instruction-tuned medical AI system that surpasses the passing threshold of 60% for the United States Medical Licensing Examination (USMLE) for the first time among all 7B-parameter models. The model was trained using [new synthetic dataset](https://huggingface.co/datasets/dmis-lab/meerkat-instructions) consisting of high-quality chain-of-thought reasoning paths sourced from 18 medical textbooks, along with diverse instruction-following datasets. This equips the model with high-level medical reasoning capabilities required for solving complex medical problems. [Nature Paper](https://www.nature.com/articles/s41746-025-01653-8)  [meerkat model](https://huggingface.co/dmis-lab/meerkat-7b-v1.0)

# Instruction-Tuning Llama-2-7b-chat on Meerkat dataset

In this notebook we will:

1. Reset the kernel and Import required libraries (and optionally fix CUDA OOM issues).  
2. Load & merge the nine Meerkat JSONL files into one `Dataset`.  
3. Transform each example into the `<s>[INST]…[/INST]…</s>` template.  
4. Split into train/eval.  
5. Initialize tokenizer, 4-bit quantized model, and LoRA PEFT config.  
6. Set up `SFTTrainer` and `TrainingArguments`.  
7. Train and save as **Llama-2-7b-chat-meerkat**.  


Reset the kernel and check the memory

In [ ]:
!nvidia-smi

Thu May  8 01:29:38 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:07:00.0 Off |                    0 |
| N/A   31C    P0              76W / 400W |      7MiB / 81920MiB |     10%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
# 1) Imports + optional CUDA memory workaround
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

# If you hit CUDA OOM, uncomment this line:
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64"

import warnings
warnings.filterwarnings("ignore")                    # ignore Python warnings

## 2a) download meerkat dataset
from hugginface


In [ ]:
%time !python3 /content/source_code/Llama2/download-meerkat-ds.py

[skip] chatdoctor-cleaned.jsonl already exists
[skip] liveqa.jsonl already exists
[skip] medbooks-18-cot.jsonl already exists
[skip] medicationqa.jsonl already exists
[skip] medinstruct-52k.jsonl already exists
[skip] medmcqa.jsonl already exists
[skip] medqa-cot.jsonl already exists
[skip] medqa-dialog.jsonl already exists
[skip] mts-dialog.jsonl already exists
✅ All files downloaded to /workspace/data_meerkat
CPU times: user 7.72 ms, sys: 7.21 ms, total: 14.9 ms
Wall time: 632 ms


## 2b) Load & merge the Meerkat JSONL files

We point at `../../data_meerkat`, find all `.jsonl` files, and load them into a single `DatasetDict` under the `"train"` split.

In [ ]:
# adjust this path if needed
meerkat_dir = "/content/data_meerkat"

# collect all JSONL paths
files = [
    os.path.join(meerkat_dir, fname)
    for fname in os.listdir(meerkat_dir)
    if fname.endswith(".jsonl")
]

# load into one big 'train' split
raw = load_dataset("json", data_files={"train": files})
raw_ds = raw["train"]

print(f"Loaded {len(raw_ds):,} total examples.")


Loaded 441,034 total examples.


## 3) Transform examples into the `<s>[INST]…[/INST]…</s>` format

We handle the various Meerkat schemas (`instruction`/`output`, `question`/`answer` + optional CoT `explanation` + `options`) and collapse each into a single string field `"text"`.


In [ ]:
def transform_meerkat(example):
    # 3a) if instruction/output format
    if "instruction" in example and "output" in example:
        prompt = example["instruction"].strip()
        response = example["output"].strip()
    # 3b) if QA + CoT format
    elif "question" in example and "answer" in example:
        explan = example.get("explanation", "").strip()
        # final answer sentence
        final = f"{explan} Therefore, the answer is {example['answer']}."
        # prepend any multiple‐choice options
        opts = example.get("options")
        if isinstance(opts, dict):
            optstr = " ".join(f"({k}) {v}" for k, v in opts.items())
            prompt = example["question"].strip() + " Options: " + optstr
        else:
            prompt = example["question"].strip()
        response = final
    else:
        # fallback to text + answer
        prompt = example.get("text", "").strip()
        response = example.get("answer", "").strip()

    return {"text": f"<s>[INST] {prompt} [/INST] {response} </s>"}

# apply and drop old columns
ds = raw_ds.map(transform_meerkat, remove_columns=raw_ds.column_names)
print("Transformed to template, example:")
print(ds[0]["text"])


Transformed to template, example:
<s>[INST]  [/INST]  </s>


## 4) Train/Validation Split

We hold out 5% of the data for evaluation.


In [ ]:
split = ds.train_test_split(test_size=0.05, seed=42)
train_dataset = split["train"]
eval_dataset  = split["test"]

print(f"→ train size: {len(train_dataset):,}")
print(f"→ eval  size: {len(eval_dataset):,}")


→ train size: 418,982
→ eval  size: 22,052


## 5) Initialize Tokenizer & Model + LoRA+4-bit Quant

- **Tokenizer** from `Llama-2-7b-chat-hf`  
- **4-bit quant** (NF4) for the model to reduce VRAM.  



In [ ]:
base_model = "../../model/Llama-2-7b-chat-hf"

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 4-bit quant config
quant_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# load quantized model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_cfg,
    device_map="auto",
)
model.config.use_cache = False
model.config.pretraining_tp = 1



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
!nvidia-smi

Thu May  8 01:30:19 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:07:00.0 Off |                    0 |
| N/A   31C    P0              69W / 400W |   4629MiB / 81920MiB |      3%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

## 6) Set up TrainingArguments & SFTTrainer

- **LoRA PEFT** config r=64, α=16, dropout=0.1.
We mirror your previous hyperparameters:  
- 2 epochs, batch_size=1, LR=2e-4, save/log every 25 steps, no fp16/bf16, constant LR schedule, TensorBoard logging.


In [ ]:
# LoRA config
peft_cfg = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_args = TrainingArguments(
    output_dir="../../model/results-meerkat",
    num_train_epochs=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    warmup_ratio=0.03,
    learning_rate=2e-4,
    optim="paged_adamw_32bit",
    max_grad_norm=0.3,
    fp16=False,
    bf16=False,
    save_steps=25,
    logging_steps=25,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    peft_config=peft_cfg,
    args=training_args,
    max_seq_length=512,
    packing=False,
)


Map:   0%|          | 0/22052 [00:00<?, ? examples/s]

## 7) Train & Save the Fine-Tuned Model

This will produce **Llama-2-7b-chat-meerkat** under `../../model/`.


In [ ]:
trainer.train()

# save under new name
out_model = "../../model/Llama-2-7b-chat-meerkat"
%time trainer.model.save_pretrained(out_model)
trainer.tokenizer.save_pretrained(out_model)

print(f"✅ Fine-tuned model saved to {out_model}")


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,2.720400
50,0.290700
75,0.147700
100,0.173400
125,0.188300
150,0.166700
175,0.152600
200,0.144000
225,0.145900


Checkpoint destination directory ../../model/results-meerkat/checkpoint-25 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ../../model/results-meerkat/checkpoint-50 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ../../model/results-meerkat/checkpoint-75 already exists and is non-empty.Saving will proceed but saved results may be invalid.


---
## References

- https://ai.meta.com/research/publications/llama-2-open-foundation-and-fine-tuned-chat-models/
- https://llama.meta.com/llama2
- https://huggingface.co/tasks/text-generation
- https://arxiv.org/abs/2304.07327
- https://huggingface.co/datasets/timdettmers/openassistant-guanaco
- https://huggingface.co/docs/transformers/en/model_doc/llama2
- https://huggingface.co/docs/peft/main/en/developer_guides/quantization

## Licensing

Copyright © 2022 OpenACC-Standard.org. This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0). These materials include references to hardware and software developed by other entities; all applicable licensing and copyrights apply.